<div class="alert alert-block alert-info">
    <b> Running URL: </b>
    </div> 

In [21]:
from urllib.request import urlopen
from json import loads
url = "https://educationdata.urban.org/api/v1/schools/crdc/discipline/2013/disability/sex/"
response = urlopen(url)
data = loads(response.read());

<div class="alert alert-block alert-success">
    Success!
    </div> 

<div class="alert alert-block alert-info">
    <b> First Code Block: </b>
    </div> 

In [2]:
# ### 
# Sanha start a .py or .R script in the repo that:
# Reads in the “Data” sheets of the following 2013-2014 CRDC files (probably want to write a function that takes in a pathname and iterate over pathnames):
# 03 - enrollment
# 10- 15
# Replaces missing values with NA (check the CRDC codebook but missing might be indicating redacted numbers)
# Replaces suppressed values with the max value pre suppression (check the CRDC codebook)
# Merge the different data elements together (probably merge on `COMBOKEY’ that represents unique school ids)


In [3]:
## Basic Packages

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string
import os
import openpyxl
from datetime import datetime
from random import sample

In [4]:
## Opening Files:

# Creating directories
filename = os.getcwd() + "\\2013-14-crdc-data.zip"
extract_dir = os.getcwd() + "\\CRDC_Data"

# Making older if it doesn't exist:
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Extracting:
import shutil
shutil.unpack_archive(filename, extract_dir)

In [5]:
## Loading Data:

file_local = "CRDC_Data\\2013-14 CRDC\\School\\CRDC-collected data file for Schools\\"

In [6]:
os.getcwd()

'C:\\Users\\Sanha Tahir\\Downloads\\GeorgeTown\\Rebecca Research'

<div class="alert alert-block alert-danger">
    There's also CRDC plus ED Facts data file; is this the right directory?
    </div> 

In [7]:
# Reads in the “Data” sheets of the following 2013-2014 CRDC files (probably want to write a function that takes in a pathname and iterate over pathnames):
#* 03 - enrollment
#* 10- 15
# Excel files are loaded into a dictionary because I want to name the keys and the dataframes

file_dict = {}

list_of_docs = ["03 Enrollment.xlsx", "10 Single Sex Interscholastic Athletics.xlsx", "11-1 Corporal Punishment.xlsx",
               "11-2 Suspensions (required elements).xlsx", "11-3 Expulsions.xlsx", "12 Student Referrals and Arrests.xlsx",
               "14 Harassment and Bullying.xlsx", "15 Restraint and Seclusion.xlsx"]

list_of_dfs = []

for i in range(len(list_of_docs)):
    loca = list_of_docs[i]
    loca_address = loca
    #print(loca_address)
    tokenized_doc = loca.split()
    df_name = re.sub(r"(\d\d)(\-)?(\d)?(\_)|\(|\)|(\.xlsx)", "","_".join(tokenized_doc))
    df_name = re.sub(r"\s", "", df_name.lower())
    print(df_name)
    list_of_dfs.append(df_name)
    key = df_name
    
    print("Starting loading for: " + df_name + " at " + str(pd.Timestamp.now()))
    file_dict[key] = pd.read_excel(loca)
    print("Ending loading for: " + df_name + " at " + str(pd.Timestamp.now()))


enrollment
Starting loading for: enrollment at 2022-11-10 13:49:09.568560
Ending loading for: enrollment at 2022-11-10 13:53:18.499117
single_sex_interscholastic_athletics
Starting loading for: single_sex_interscholastic_athletics at 2022-11-10 13:53:18.516194
Ending loading for: single_sex_interscholastic_athletics at 2022-11-10 13:53:49.971428
corporal_punishment
Starting loading for: corporal_punishment at 2022-11-10 13:53:49.971428
Ending loading for: corporal_punishment at 2022-11-10 13:55:01.662391
suspensions_required_elements
Starting loading for: suspensions_required_elements at 2022-11-10 13:55:01.662391
Ending loading for: suspensions_required_elements at 2022-11-10 13:59:08.876756
expulsions
Starting loading for: expulsions at 2022-11-10 13:59:08.881756
Ending loading for: expulsions at 2022-11-10 14:04:42.176247
student_referrals_and_arrests
Starting loading for: student_referrals_and_arrests at 2022-11-10 14:04:42.179958
Ending loading for: student_referrals_and_arrests a

In [10]:
enrollment  = pd.DataFrame(file_dict["enrollment"])
single_sex_interscholastic_athletics   = pd.DataFrame(file_dict["single_sex_interscholastic_athletics"])
corporal_punishment  = pd.DataFrame(file_dict["corporal_punishment"])
suspensions_required_elements  = pd.DataFrame(file_dict["suspensions_required_elements"])
expulsions  = pd.DataFrame(file_dict["expulsions"])
student_referrals_and_arrests  = pd.DataFrame(file_dict["student_referrals_and_arrests"])
harassment_and_bullying = pd.DataFrame(file_dict["harassment_and_bullying"])
restraint_and_seclusion = pd.DataFrame(file_dict["restraint_and_seclusion"])

In [11]:
list_of_dfs = [enrollment, 
               single_sex_interscholastic_athletics, 
               corporal_punishment, 
               suspensions_required_elements, 
               student_referrals_and_arrests, 
              harassment_and_bullying,
              restraint_and_seclusion]

In [ ]:
# df_dict = {'first': df1, 'second': df2, 'third': df3}
# df_dict = {k: v.pipe(foo) for k, v in df_dict.items()}


### Codebook and details:
Codebook link: https://ocrdata.ed.gov/assets/downloads/CRDC-Public-Use-Data-File-Users-Manual-2013-14.pdf
* DSO_SCH_GTENR_AS_F is the data system overload flag for the variable SCH_GTENR_AS_F, where 1 indicates that the data element could contain errors.
* -5  = Missing value for CRDC item; user did not provide a value
* -2  = Small Cell Value i.e. Surpressed Values <br>
Most supressed values i.e. -2 are personal identifiers: <br>
                SCH_IDEAENR_ -> Students with disabilities served under IDEA <br>
                SCH_ENR_IDEA_ : Overall Student Enrollment: IDEA Male <br>
                HI_M/F: Hispanic Male/Female <br>
                AM_M/F: American Indian/Alaska Native
                AS_M/F: Asian <br>
                HP_M/F: Native Hawaiian/Pacific Islander <br>
                BL_M/F: Black <br>
                WH_M/F: White <br>
                TR_M/F: Two or more races 
                SCH_IDEAENR_LEP_ : Students with disabilities served under IDEA: LEP <br>
                TOT_IDEAENR_: Total Number of Students with a Disability, served under IDEA:  <br>
                SCH_IDEAENR_ -> Students with disabilities served under IDEA <br>
                SCH_ENR_IDEA_ : Overall Student Enrollment: IDEA Male <br>
                HI_M/F: Hispanic Male/Female <br>
                AM_M/F: American Indian/Alaska Native <br>
                AS_M/F: Asian <br>
                HP_M/F: Native Hawaiian/Pacific Islander <br>
                BL_M/F: Black <br>
                WH_M/F: White <br>
                TR_M/F: Two or more races  <br>
                SCH_IDEAENR_LEP_ : Students with disabilities served under IDEA: LEP <br>
                TOT_IDEAENR_: Total Number of Students with a Disability, served under IDEA <br>

* -9  = Missing value for CRDC item because of system skip logic: <br>
                 Skip Logic refers to questions that do not need to be answered based on answers provided for previous questions. For example, if a respondent answers that preschool wservices were not provided nu a school, any future questions pertaining to preschool would be skipped and respondants would not have to answer the, Auto zeroes is if total/overall count if call 0.
                 
## DSO Flags:
Data System Overload Anomaly
On August 8–12, 2015, and again on August 18–19, the data collection system experienced an <b> unexpectedly high
volume of users, </b> which required more processing than the data collection system was configured to handle. This
spike in usage caused a problem with the system’s error processing (see appendix A), which allowed LEAs to certify
their submission without resolving potential errors. <br> <b> This issue affected 132 LEAs, which combined had over 800
unresolved errors. </b> <br> The PSC was able to work with 18 LEAs to correct errors and reduce the number of unresolved
errors to <b> 263 </b> . Processing flags were created (see section 6.2.6) for the unresolved errors in 114 LEAs. <br> The system identified a variety of error types therefore some unresolved errors may be misreported data while others may be
correctly reported data or correctly reported data in the wrong format, for example.

### Replaces suppressed values with the max value pre suppression (check the CRDC codebook)

Surpressed values are coded -2.

In [12]:
def sum_table(df: pd.DataFrame):
    
    unique_vlaues = pd.DataFrame()
    unique_vlaues[["col_nm", "uniq_vals"]] = [[col, pd.unique(df[col])]
                                              for col in df.columns] # This is supposed to be enrollment.cols
    unique_vlaues["missing_cols"] = [-5 in xyz for xyz in unique_vlaues["uniq_vals"]]
    unique_vlaues["surpress_cols"] = [-2 in xyz for xyz in unique_vlaues["uniq_vals"]]
    unique_vlaues["skipped_cols"] = [-9 in xyz for xyz in unique_vlaues["uniq_vals"]]
    mv = len(unique_vlaues[unique_vlaues["missing_cols"] == True])
    sr = len(unique_vlaues[unique_vlaues["surpress_cols"] == True])
    sv = len(unique_vlaues[unique_vlaues["skipped_cols"] == True])
    
    stats_append = pd.DataFrame({
        "df_name": str(key),
        "missing_cols": [mv],
        "surpress_cols": [sr],
        "skipped_cols": [sv]
    })
    
    return(stats_append)
    
print("Starting function at: " + str(pd.Timestamp.now()))
print("Number of Columns With Given Type of Data:")
stats_dict = pd.DataFrame(columns = ["df_name", "missing_cols", "surpress_cols", "skipped_cols"])
for key in file_dict:
    stats_dict = pd.concat([stats_dict, sum_table(file_dict[key])])
    
print("Ending function at: " + str(pd.Timestamp.now()))
stats_dict

Starting function at: 2022-11-10 14:26:25.594947
Number of Columns With Given Type of Data:
Ending function at: 2022-11-10 14:26:27.977258


,df_name,missing_cols,surpress_cols,skipped_cols
0,enrollment,92,20,0
0,single_sex_interscholastic_athletics,9,0,9
0,corporal_punishment,38,0,38
0,suspensions_required_elements,154,0,154
0,expulsions,134,0,134
0,student_referrals_and_arrests,76,0,76
0,harassment_and_bullying,137,0,135
0,restraint_and_seclusion,123,0,123


In [13]:
def sum_each(df):
    
    total_vals = df.shape[0] * df.shape[1]
    missing_vals = 0
    surpressed_vals = 0
    skipped_vals = 0
    
    for col in df.columns:
        missing_vals =+ missing_vals + len(df[df[col] == -2])
        surpressed_vals =+ surpressed_vals + len(df[df[col] == -5])
        skipped_vals =+ skipped_vals + len(df[df[col] == -9])
        
    stats_append = pd.DataFrame({
        "df_name": str(key),
        "missing_vals": [missing_vals],
        "surpress_vals": [surpressed_vals],
        "skipped_vals": [skipped_vals],
        "total_vals": [total_vals]
    })
    
    return(stats_append)
    
print("Starting function at: " + str(pd.Timestamp.now()))

print("Total cells with incorrect/missing data:")

stats_dict2 = pd.DataFrame(columns = ["df_name", "missing_vals", "surpress_vals", "skipped_vals", "total_vals"])
for key in file_dict:
    stats_dict2 = pd.concat([stats_dict2, sum_each(file_dict[key])])

stats_dict2["missing_portion"] = stats_dict2["missing_vals"]/stats_dict2["total_vals"]
stats_dict2["surpressed_portion"] = stats_dict2["surpress_vals"]/stats_dict2["total_vals"]
stats_dict2["skipped_portion"] = stats_dict2["skipped_vals"]/stats_dict2["total_vals"]

print("Ending function at: " + str(pd.Timestamp.now()))

stats_dict2

Starting function at: 2022-11-10 14:26:28.019413
Total cells with incorrect/missing data:
Ending function at: 2022-11-10 14:26:38.621420


,df_name,missing_vals,surpress_vals,skipped_vals,total_vals,missing_portion,surpressed_portion,skipped_portion
0,enrollment,1160860,1077,0,15281120,0.075967,0.00007,0.0
0,single_sex_interscholastic_athletics,0,621,714042,1719126,0.0,0.000361,0.415352
0,corporal_punishment,0,727,3428902,4393322,0.0,0.000165,0.78048
0,suspensions_required_elements,0,3949,2874608,15376627,0.0,0.000257,0.186947
0,expulsions,0,12889,1543599,22062117,0.0,0.000584,0.069966
0,student_referrals_and_arrests,0,77349,171194,7927081,0.0,0.009758,0.021596
0,harassment_and_bullying,0,100119,221607,14039529,0.0,0.007131,0.015785
0,restraint_and_seclusion,0,40680,206067,12606924,0.0,0.003227,0.016346


# Cleaning values:

1. Missing values: From -2 to NA
2. Surpressed values: From -5 to ??
3. Skipped values: From -9 to 0

In [14]:
print("Checking for missing values before replacement: " + str(file_dict["enrollment"].at[0, "SCH_IDEAENR_AM_M"]))
print("Checking for surpressed values before replacement: " + str(file_dict["single_sex_interscholastic_athletics"].at[0, "SCH_SSATHLETICS_IND"]))

for key in file_dict:
    file_dict[key] = file_dict[key].replace(-2, None)
    file_dict[key] = file_dict[key].replace("-9", 0)
    
print("Checking for missing values after replacement: " + str(file_dict["enrollment"].at[0, "SCH_IDEAENR_AM_M"]))
print("Checking for surpressed values after replacement: " + str(file_dict["single_sex_interscholastic_athletics"].at[0, "SCH_SSATHLETICS_IND"]))

Checking for missing values before replacement: -2
Checking for surpressed values before replacement: -9
Checking for missing values after replacement: None
Checking for surpressed values after replacement: 0


# Merging:

In [15]:
# list_of_dfs = [enrollment, 
#                single_sex_interscholastic_athletics, 
#                corporal_punishment, 
#                suspensions_required_elements, 
#                student_referrals_and_arrests, 
#               harassment_and_bullying,
#               restraint_and_seclusion]

en_ssia = enrollment.merge(single_sex_interscholastic_athletics, how = "inner",
                           right_on = "COMBOKEY", left_on = "COMBOKEY")

In [16]:
print("Length of enrollment: " + str(len(enrollment.index)))
print("Length of ssia: " + str(len(single_sex_interscholastic_athletics.index)))
print("Length of merge: " + str(len(en_ssia.index)))

Length of enrollment: 95507
Length of ssia: 95507
Length of merge: 95507


<div class="alert alert-block alert-success">
    Merge on COMBOKEY seems be a complete inner join.
    </div> 

In [17]:
en_ssia[en_ssia["JJ_x"] != en_ssia["JJ_y"]]

,LEA_STATE_x,LEAID_x,LEA_NAME_x,SCHID_x,SCH_NAME_x,COMBOKEY,JJ_x,SCH_ENR_HI_M,SCH_ENR_HI_F,SCH_ENR_AM_M,...,SCH_SSSPORTS_M,SCH_SSSPORTS_F,TOT_SSSPORTS,SCH_SSTEAMS_M,SCH_SSTEAMS_F,TOT_SSTEAMS,SCH_SSPART_M,SCH_SSPART_F,TOT_SSPART,DSO_SCH_SSATHLETICS_IND


In [18]:

# for df in list_of_dfs:
for key in file_dict:
    # df.pipe(sum_table)
    stats_dict = pd.concat([stats_dict, sum_table(file_dict[key])])
    
print("Ending function at: " + str(pd.Timestamp.now()))
stats_dict

Ending function at: 2022-11-10 14:26:44.073400


,df_name,missing_cols,surpress_cols,skipped_cols
0,enrollment,92,20,0
0,single_sex_interscholastic_athletics,9,0,9
0,corporal_punishment,38,0,38
0,suspensions_required_elements,154,0,154
0,expulsions,134,0,134
0,student_referrals_and_arrests,76,0,76
0,harassment_and_bullying,137,0,135
0,restraint_and_seclusion,123,0,123
0,enrollment,92,0,0
0,single_sex_interscholastic_athletics,9,0,9


<div class="alert alert-block alert-success">
    No mismatch in joined columns of same name.
    </div> 

In [19]:
sra_ssia = student_referrals_and_arrests.merge(single_sex_interscholastic_athletics, how = "inner",
                           right_on = "COMBOKEY", left_on = "COMBOKEY")
print("Length of single_sex_interscholastic_athletics: " + str(len(student_referrals_and_arrests.index)))
print("Length of student_referrals_and_arrests: " + str(len(single_sex_interscholastic_athletics.index)))
print("Length of merge: " + str(len(sra_ssia.index)))
sra_ssia[sra_ssia["JJ_x"] != sra_ssia["JJ_y"]]

Length of single_sex_interscholastic_athletics: 95507
Length of student_referrals_and_arrests: 95507
Length of merge: 95507


,LEA_STATE_x,LEAID_x,LEA_NAME_x,SCHID_x,SCH_NAME_x,COMBOKEY,JJ_x,SCH_DISCWODIS_REF_HI_M,SCH_DISCWODIS_REF_HI_F,SCH_DISCWODIS_REF_AM_M,...,SCH_SSSPORTS_M,SCH_SSSPORTS_F,TOT_SSSPORTS,SCH_SSTEAMS_M,SCH_SSTEAMS_F,TOT_SSTEAMS,SCH_SSPART_M,SCH_SSPART_F,TOT_SSPART,DSO_SCH_SSATHLETICS_IND
